### **Kbank Google Playstore App Crawler**
- Contributor: Jihyun Jung
- References
    - Tistory Blog: https://heytech.tistory.com/293
- Last Updated@2022-08-07

<img src="https://m.jungle.co.kr/image/51157065426dbb7b55877a01" width="350" height="200">

### 초기 설정

In [1]:
chrome_driver = '../chromedriver'

In [2]:
url = 'https://play.google.com/store/apps/details?id=com.kbankwith.smartbank'

In [3]:
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd

### 무한 스크롤 함수

In [4]:
def scroll(modal):
    try:
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            time.sleep(pause_time)
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)

            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height

    except Exception as e:
        print("에러 발생: ", e)

### 데이터 크롤링

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-setuid-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option('escludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(executable_path='../chromedriver.exe')

driver.get(url)
wait = WebDriverWait(driver, 13)

all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element_by_xpath(all_review_button_xpath).click()

all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

C:\Users\82108\AppData\Local\Temp\ipykernel_2584\3053915192.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../chromedriver.exe')
C:\Users\82108\AppData\Local\Temp\ipykernel_2584\3053915192.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(all_review_button_xpath).click()
C:\Users\82108\AppData\Local\Temp\ipykernel_2584\3219276747.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()


스크롤 완료


### HTML Parsing

In [6]:
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

### 크롬 드라이버 종료

In [7]:
driver.quit()

### HTML 데이터 저장

In [8]:
with open("../dataset/data_html.html", "w", encoding='utf-8') as file:
    file.write(str(soup_source))

### 데이터프레임 변환

In [9]:
review_source = soup_source.find_all(class_ = 'RHo1pe')

dataset = []
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in review_source:
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

### 데이터프레임 저장

In [10]:
df = pd.DataFrame(dataset)
df.to_csv('../dataset/review_dataset.csv', encoding='utf-8-sig')

### 리뷰 데이터 불러오기

In [11]:
df = pd.read_csv('../dataset/review_dataset.csv', encoding='utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1)
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220601,2022,6,1,1,동동,타 기관 인증서 불러오기가 안되는데 대체 무슨 일이죠?? 범용공동인증서를 등록하려고...
1,2,20220626,2022,6,26,2,Younghoon Jeon,다른건 다 모르겠고 어플 켰을 때 메인 화면이 뜨기까지 시간 좀 더 단축시켜주세요....
2,3,20220629,2022,6,29,1,Kuee Haeng Lee,타기관인증서를 갱신하면서 인증서가져오기를 수행했는데요. QR로 가져오기는 아예 안돼...
3,4,20220709,2022,7,9,3,brian lryan,갤럭시쓰는데 알림진동이 안 울립니다.(에티켓모드 아님) 전에 리뷰보니까 다른분들도 ...
4,5,20220625,2022,6,25,5,박태준,뭣같은 앱아이콘을 바꾼건 정말 좋네요. 거기다 묘하게 앱 자체의 속도도 빨라진 느낌...
...,...,...,...,...,...,...,...,...
2035,2036,20210506,2021,5,6,1,사람이 꽃보다 아름다워,갤럭시s9+사용 고객입니다.. 케이뱅크 앱이 실행이 안되고 로고만 뜨네요.. 3일내...
2036,2037,20210816,2021,8,16,1,Call Me K,저는 당신들이 너무 싫습니다. 잘쓰고 있던거를 못쓰게 하고 돈도 못보내고 정말 당신...
2037,2038,20210223,2021,2,23,1,아주그냥,접속을 못하네 인출좀 할려고 하니 상담원은 계속 통화중... 지친다
2038,2039,20210413,2021,4,13,1,zin lee,인증번호자체가 안오네요 삼성갤럭시 S9 최신OS 시간은 오후 8시부터 계속~ 쭉이요...
